In [1]:
# !pip install pymupdf
# !pip install easyocr
import pandas as pd
import fitz #PyMuPDF
import io
import os
import PIL
from PIL import Image
from PIL import ImageDraw
import easyocr

reader = easyocr.Reader(['en'], gpu = False)

Using CPU. Note: This module is much faster with a GPU.


In [2]:
def extract_images_from_pdf(pdf_file, path) :
    with fitz.open(path+"\\"+pdf_file) as my_pdf_file:
        table = []
        for page_number in range(1, len(my_pdf_file)+1):
            new_table = []
            page = my_pdf_file[page_number-1]
            images = page.getImageList()
            if images:
                print(f"There are {len(images)} image/s on page number {page_number}[+]")
            else:
                print(f"There are No image/s on page number {page_number}[!]")
            for image_number, image in enumerate(page.getImageList(), start=1):
                xref_value = image[0]
                base_image = my_pdf_file.extractImage(xref_value)
                image_bytes = base_image["image"]
                ext = base_image["ext"]
                image = Image.open(io.BytesIO(image_bytes))
#                 image.save(open(f"Images/Page{page_number}Image{image_number}.{ext}", "wb"))
                table.append(image_to_table(image, new_table))
    return table

def draw_boxes(image, bounds, color='yellow', width=2):
    draw = ImageDraw.Draw(image)
    for bound in bounds:
        p0, p1, p2, p3 = bound[0]
        draw.line([*p0, *p1, *p2, *p3, *p0], fill=color, width=width)
    return image

def image_to_table(image, table) :
#     image = PIL.Image.open(img_path)
    bounds = reader.readtext(image)
    draw_boxes(image, bounds)

    table.append([bounds[0][0][0][1], bounds[0][1]])
    for b in range(1,len(bounds)):
        if (bounds[b][0][0][1] - bounds[b-1][0][0][1]) < 20 :
            bounds[b][0][0][1] = bounds[b-1][0][0][1]
            table.append([bounds[b-1][0][0][1], bounds[b][1]])
        else :
            table.append([bounds[b][0][0][1], bounds[b][1]])
            
    values = sorted(set(map(lambda x:x[0], table)))
    new_table = [[y[1] for y in table if y[0]==x] for x in values]
    return new_table

def table_to_excel(table, pdf_file) :
    df= [item for sublist in table for item in sublist]
    df_new = pd.DataFrame(df)
    df_new.to_excel("Excel//"+ pdf_file[:-4] +"_excel.xlsx", header=False, index=False)
    return df_new

In [3]:
path = os.path.abspath(os.getcwd())
os.mkdir(path + '/Excel')
pdfs_folder = 'pdfs' #modify the folder name
pdf_list = os.listdir(path+"/"+pdfs_folder)
path = path+"\\"+pdfs_folder

for file in pdf_list :
    tables = extract_images_from_pdf(file, path)
    table_to_excel(tables, file)

Deprecation: 'getImageList' removed from class 'Page' after v1.19 - use 'get_images'.
Deprecation: 'extractImage' removed from class 'Document' after v1.19 - use 'extract_image'.


There are 1 image/s on page number 1[+]
There are 1 image/s on page number 1[+]
There are 1 image/s on page number 2[+]
